In [151]:
# import resources
import numpy as np
import torch

# random seed (for reproducibility)
seed = 1
# set random seed for numpy
np.random.seed(seed)
# set random seed for pytorch
torch.manual_seed(seed)

In [152]:
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# convert data to Tensors
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.CIFAR10(root='data', train=True,
                            download=True, transform=transform)

test_data = datasets.CIFAR10(root='data', train=False, 
                           download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, 
                                           batch_size=batch_size, 
                                           num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(test_data, 
                                          batch_size=batch_size, 
                                          num_workers=num_workers)

Files already downloaded and verified
Files already downloaded and verified


In [153]:
import matplotlib.pyplot as plt
%matplotlib inline
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = torch.from_numpy(images.numpy())

# # plot the images in the batch, along with the corresponding labels
# fig = plt.figure(figsize=(25, 4))
# for idx in np.arange(batch_size):
#     ax = fig.add_subplot(2, batch_size/2, idx+1, xticks=[], yticks=[])
#     ax.imshow(np.squeeze(images[idx]), cmap='gray')
#     # print out the correct label for each image
#     # .item() gets the value contained in a Tensor
#     ax.set_title(str(labels[idx].item()))

In [154]:
import torch.nn as nn
import torch.nn.functional as F

In [155]:
# Initial convolutional layer: we're running 256 kernels of size 9x9 on the image
# This generates one 24x24 (if using CIFAR) feature map with 256 channels 
# https://datascience.stackexchange.com/questions/64278/what-is-a-channel-in-a-cnn

class ConvLayer(nn.Module):
    
    def __init__(self, in_channels, out_channels=256, kernel_size=9, stride=1, padding=0):
        '''Constructs the ConvLayer with a specified input and output size.
            param in_channels: input depth of an image, default value = 1
            param out_channels: output depth of the convolutional layer, default value = 256
            param kernel_size: size of the convolutional kernel, default value = 9
            param stride: stride of the convolutional kernel, default value = 1
            param padding: zero padding added to the input, default value = 0
           '''
        super(ConvLayer, self).__init__()

        # defining a convolutional layer of the specified size
        self.conv = nn.Conv2d(in_channels, out_channels, 
                              kernel_size=kernel_size, stride=stride, padding=padding)

    def forward(self, x):
        '''Defines the feedforward behavior.
           param x: the input to the layer; an input image
           return: a relu-activated, convolutional layer
           '''
        # applying a ReLu activation to the outputs of the conv layer
        raw_output = self.conv(x)
        features = F.relu(raw_output) # will have dimensions (batch_size, n - 8, n - 8, 256), where n is the size of the original image
        return features
    

In [156]:
class PrimaryCaps(nn.Module):
    
    def __init__(self, num_capsules=8, in_channels=256, out_channels=32, kernel_size=9, stride=2, padding=0):
        '''Constructs a list of convolutional layers to be used in 
           creating capsule output vectors.
            param num_capsules: number of capsules to create
            param in_channels: input depth of features, default value = 256
            param out_channels: output depth of the convolutional layers, default value = 32
            param kernel_size: size of the convolutional kernel, default value = 9
            param stride: stride of the convolutional kernel, default value = 2
            param padding: zero padding added to the input, default value = 0
           '''
        super(PrimaryCaps, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.out_channels = out_channels
        self.padding = padding
        # creating a list of convolutional layers for each capsule I want to create
        # all capsules have a conv layer with the same parameters
        self.capsules = nn.ModuleList([
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                      kernel_size=kernel_size, stride=stride, padding=padding)
            for _ in range(num_capsules)])
    
    def forward(self, x):
        '''Defines the feedforward behavior.
           param x: the input; features from a convolutional layer
           return: a set of normalized, capsule output vectors
           '''
        # get batch size of inputs
        batch_size = x.shape[0]
  
        # the capsule network array ouptuts batch_size number of feature maps that are
        # (n - (kernel_size - 1)) / stride by (n - (kernel_size - 1)) / stride
        # and have out_channels number of channels
        
        # reshape the the capsule net array outputs to be (batch_size, TOTAL_NUMBERS_IN_FEATURE_MAP, 1)
        # this total number of numbers in feature map is (n - (kernel_size - 1)) / stride * (n - (kernel_size - 1)) / stride * out_channels
        # THE ABOVE NUMBER MUST BE EDITED IF PADDING IS CHANGED

        # in all of this, n is the dimensions of x
        # the purpose of this is to flatten the feature map into a single 1d vector
        total_params = (
            (x.shape[3] - (self.kernel_size - 1) + self.padding * 2) / self.stride * 
            (x.shape[2] - (self.kernel_size - 1) + self.padding * 2) / self.stride * 
            self.out_channels
        )

        assert total_params.is_integer()

        total_params = int(total_params)

        u = [capsule(x).view(batch_size, total_params, 1) for capsule in self.capsules]
        # stack up output vectors, u, one for each capsule
        u = torch.cat(u, dim=-1)
        # squashing the stack of vectors
        u_squash = self.squash(u)
        return u_squash, total_params
    
    def squash(self, input_tensor):
        '''Squashes an input Tensor so it has a magnitude between 0-1.
           param input_tensor: a stack of capsule inputs, s_j
           return: a stack of normalized, capsule output vectors, v_j 
           '''
        squared_norm = (input_tensor ** 2).sum(dim=-1, keepdim=True)
        scale = squared_norm / (1 + squared_norm) # normalization coeff
        output_tensor = scale * input_tensor / torch.sqrt(squared_norm)    
        return output_tensor
    

In [157]:
import helpers # to get transpose softmax function

# dynamic routing
def dynamic_routing(b_ij, u_hat, squash, routing_iterations=3):
    '''Performs dynamic routing between two capsule layers.
       param b_ij: initial log probabilities that capsule i should be coupled to capsule j
       param u_hat: input, weighted capsule vectors, W u
       param squash: given, normalizing squash function
       param routing_iterations: number of times to update coupling coefficients
       return: v_j, output capsule vectors
       '''    
    # update b_ij, c_ij for number of routing iterations
    for iteration in range(routing_iterations):
        # softmax calculation of coupling coefficients, c_ij
        c_ij = helpers.softmax(b_ij, dim=2)

        # calculating total capsule inputs, s_j = sum(c_ij*u_hat)
        s_j = (c_ij * u_hat).sum(dim=2, keepdim=True)

        # squashing to get a normalized vector output, v_j
        v_j = squash(s_j)

        # if not on the last iteration, calculate agreement and new b_ij
        if iteration < routing_iterations - 1:
            # agreement
            a_ij = (u_hat * v_j).sum(dim=-1, keepdim=True)
            
            # new b_ij
            b_ij = b_ij + a_ij
    
    return v_j # return latest v_j
    

In [158]:
# it will also be relevant, in this model, to see if I can train on gpu
TRAIN_ON_GPU = torch.cuda.is_available()

if(TRAIN_ON_GPU):
    print('Training on GPU!')
else:
    print('Only CPU available')
    

Only CPU available


In [159]:
# RENAME TO FinalCaps
# THIS CLASS SHOULD HAVE NUM_CAPSULES = TO THE ACTUAL NUMBER OF CLASSES IN THE CLASSIFICATION PROBLEM
# HAVE THIS BE AN INPUT VARIABLE AND CHANGE DECODER CLASS ACCORDINGLY
# cifar 10 has 10 classes so all good


class DigitCaps(nn.Module):
    
    def __init__(self, num_capsules=10, 
                 in_channels=8, out_channels=16):
        '''Constructs an initial weight matrix, W, and sets class variables.
           param num_capsules: number of capsules to create
           param previous_layer_nodes: dimension of input capsule vector, default value = 1152
           param in_channels: number of capsules in previous layer, default value = 8
           param out_channels: dimensions of output capsule vector, default value = 16
           '''
        super(DigitCaps, self).__init__()

        # setting class variables
        self.num_capsules = num_capsules
        self.in_channels = in_channels # previous layer's number of capsules
        self.out_channels = out_channels
        self.W = None
        self.previous_layer_nodes = None

    def forward(self, u, previous_layer_nodes):
        '''Defines the feedforward behavior.
           param u: the input; vectors from the previous PrimaryCaps layer
           return: a set of normalized, capsule output vectors
           '''

        if self.W is None:
            self.previous_layer_nodes = previous_layer_nodes
            # starting out with a randomly initialized weight matrix, W
            # these will be the weights connecting the PrimaryCaps and DigitCaps layers
            self.W = nn.Parameter(torch.randn(self.num_capsules, previous_layer_nodes, 
                                            self.in_channels, self.out_channels))
        
        # adding batch_size dims and stacking all u vectors
        u = u[None, :, :, None, :]
        # 4D weight matrix
        W = self.W[:, None, :, :, :]
        
        # calculating u_hat = W*u
        u_hat = torch.matmul(u, W)

        # getting the correct size of b_ij
        # setting them all to 0, initially
        b_ij = torch.zeros(*u_hat.size())
        
        # moving b_ij to GPU, if available
        if TRAIN_ON_GPU:
            b_ij = b_ij.cuda()

        # update coupling coefficients and calculate v_j
        v_j = dynamic_routing(b_ij, u_hat, self.squash, routing_iterations=3)

        return v_j # return final vector outputs
    
    
    def squash(self, input_tensor):
        '''Squashes an input Tensor so it has a magnitude between 0-1.
           param input_tensor: a stack of capsule inputs, s_j
           return: a stack of normalized, capsule output vectors, v_j
           '''
        # same squash function as before
        squared_norm = (input_tensor ** 2).sum(dim=-1, keepdim=True)
        scale = squared_norm / (1 + squared_norm) # normalization coeff
        output_tensor = scale * input_tensor / torch.sqrt(squared_norm)    
        return output_tensor
    

In [160]:
class RGBSigmoid(nn.Module):
    def forward(self, x):
        return torch.sigmoid(x) * 255

class Decoder(nn.Module):
    
    def __init__(self, num_channels, height, width, input_vector_length=16, input_capsules=10, hidden_dim=512):
        '''Constructs an series of linear layers + activations.
           param input_vector_length: dimension of input capsule vector, default value = 16
           param input_capsules: number of capsules in previous layer, default value = 10
           param hidden_dim: dimensions of hidden layers, default value = 512
           '''
        super(Decoder, self).__init__()
        
        # calculate input_dim
        input_dim = input_vector_length * input_capsules
        
        self.num_channels = num_channels
        self.height = height
        self.width = width

        if num_channels == 3:
            # define linear layers + activations
            self.linear_layers = nn.Sequential(
                nn.Linear(input_dim, hidden_dim), # first hidden layer
                nn.ReLU(inplace=True),
                nn.Linear(hidden_dim, hidden_dim*2), # second, twice as deep
                nn.ReLU(inplace=True),
                nn.Linear(hidden_dim*2, num_channels * height * width), 
                RGBSigmoid() # scaled sigmoid to get values from 0-255
                )
        else:
            self.linear_layers = nn.Sequential(
                nn.Linear(input_dim, hidden_dim), # first hidden layer
                nn.ReLU(inplace=True),
                nn.Linear(hidden_dim, hidden_dim*2), # second, twice as deep
                nn.ReLU(inplace=True),
                nn.Linear(hidden_dim*2, num_channels * height * width), 
                nn.Sigmoid() # scaled sigmoid to get values from 0-255
                )
        
        
    def forward(self, x):
        '''Defines the feedforward behavior.
           param x: the input; vectors from the previous DigitCaps layer
           return: two things, reconstructed images and the class scores, y
           '''
        classes = (x ** 2).sum(dim=-1) ** 0.5
        classes = F.softmax(classes, dim=-1)
        
        # find the capsule with the maximum vector length
        # here, vector length indicates the probability of a class' existence
        _, max_length_indices = classes.max(dim=1)
        
        # create a sparse class matrix
        sparse_matrix = torch.eye(10) # 10 is the number of classes
        if TRAIN_ON_GPU:
            sparse_matrix = sparse_matrix.cuda()
        # get the class scores from the "correct" capsule
        y = sparse_matrix.index_select(dim=0, index=max_length_indices.data)
        
        # create reconstructed pixels
        x = x * y[:, :, None]
        # flatten image into a vector shape (batch_size, vector_dim)
        flattened_x = x.contiguous().view(x.shape[0], -1)
        # create reconstructed image vectors
        reconstructions = self.linear_layers(flattened_x)
        
        # return reconstructions and the class scores, y
        return reconstructions, y

In [161]:
class CapsuleNetwork(nn.Module):
    def __init__(self, images):
        '''Constructs a complete Capsule Network.'''
        super(CapsuleNetwork, self).__init__()
        self.images = images
        print("image shape hting", images.shape[1])
        self.conv_layer = ConvLayer(images.shape[1])
        self.primary_capsules = PrimaryCaps()
        self.digit_capsules = DigitCaps()
        self.decoder = Decoder(images.shape[1], images.shape[2], images.shape[3])
                
    def forward(self):
        '''Defines the feedforward behavior.
           param images: the original CIFAR10 image input data
           return: output of DigitCaps layer, reconstructed images, class scores
           '''
        primary_caps_output, total_params = self.primary_capsules(self.conv_layer(self.images))
        caps_output = self.digit_capsules(primary_caps_output, total_params).squeeze().transpose(0,1)
        reconstructions, y = self.decoder(caps_output)
        return caps_output, reconstructions, y
    

In [162]:
# instantiate and print net
capsule_net = CapsuleNetwork(images)

# move model to GPU, if available 
if TRAIN_ON_GPU:
    capsule_net = capsule_net.cuda()

image shape hting 3


In [163]:
class CapsuleLoss(nn.Module):
    
    def __init__(self):
        '''Constructs a CapsuleLoss module.'''
        super(CapsuleLoss, self).__init__()
        self.reconstruction_loss = nn.MSELoss(reduction='sum') # cumulative loss, equiv to size_average=False

    def forward(self, x, labels, images, reconstructions):
        '''Defines how the loss compares inputs.
           param x: digit capsule outputs
           param labels: 
           param images: the original CIFAR10 image input data
           param reconstructions: reconstructed CIFAR10 image data
           return: weighted margin and reconstruction loss, averaged over a batch
           '''
        batch_size = x.shape[0]

        ##  calculate the margin loss   ##
        
        # get magnitude of digit capsule vectors, v_c
        v_c = torch.sqrt((x**2).sum(dim=2, keepdim=True))

        # calculate "correct" and incorrect loss
        left = F.relu(0.9 - v_c).view(batch_size, -1)
        right = F.relu(v_c - 0.1).view(batch_size, -1)
        
        # sum the losses, with a lambda = 0.5
        margin_loss = labels * left + 0.5 * (1. - labels) * right
        margin_loss = margin_loss.sum()
        ##  calculate the reconstruction loss   ##
        images = images.view(reconstructions.size()[0], -1)
        reconstruction_loss = self.reconstruction_loss(reconstructions, images)

        # return a weighted, summed loss, averaged over a batch size
        return (margin_loss + 0.0005 * reconstruction_loss) / images.size(0)


In [164]:
import torch.optim as optim

# custom loss
criterion = CapsuleLoss()

# Adam optimizer with default params
optimizer = optim.Adam(capsule_net.parameters())

In [165]:
def train(capsule_net, criterion, optimizer, 
          n_epochs, print_every=300):
    '''Trains a capsule network and prints out training batch loss statistics.
       Saves model parameters if *validation* loss has decreased.
       param capsule_net: trained capsule network
       param criterion: capsule loss function
       param optimizer: optimizer for updating network weights
       param n_epochs: number of epochs to train for
       param print_every: batches to print and save training loss, default = 100
       return: list of recorded training losses
       '''

    # track training loss over time
    losses = []

    # one epoch = one pass over all training data 
    for epoch in range(1, n_epochs+1):

        # initialize training loss
        train_loss = 0.0
        
        capsule_net.train() # set to train mode
    
        # get batches of training image data and targets
        for batch_i, (images, target) in enumerate(train_loader):

            # reshape and get target class
            target = torch.eye(10).index_select(dim=0, index=target)

            if TRAIN_ON_GPU:
                images, target = images.cuda(), target.cuda()

            # zero out gradients
            optimizer.zero_grad()
            # get model outputs
            caps_output, reconstructions, y = capsule_net()
            # calculate loss
            loss = criterion(caps_output, target, images, reconstructions)
            # perform backpropagation and optimization
            loss.backward()
            optimizer.step()

            train_loss += loss.item() # accumulated training loss
            
            # print and record training stats
            if batch_i != 0 and batch_i % print_every == 0:
                avg_train_loss = train_loss/print_every
                losses.append(avg_train_loss)
                print('Epoch: {} \tTraining Loss: {:.8f}'.format(epoch, avg_train_loss))
                train_loss = 0 # reset accumulated training loss
        
    return losses
    

In [166]:
# training for 3 epochs
n_epochs = 1
losses = train(capsule_net, criterion, optimizer, n_epochs=n_epochs)

Epoch: 1 	Training Loss: 757.31782843
Epoch: 1 	Training Loss: 0.97462035
Epoch: 1 	Training Loss: 0.96249412


KeyboardInterrupt: 

In [167]:
plt.plot(losses)
plt.title("Training Loss")
plt.show()

NameError: name 'losses' is not defined

In [168]:
def test(capsule_net, test_loader):
    '''Prints out test statistics for a given capsule net.
       param capsule_net: trained capsule network
       param test_loader: test dataloader
       return: returns last batch of test image data and corresponding reconstructions
       '''
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    
    test_loss = 0 # loss tracking

    capsule_net.eval() # eval mode

    for batch_i, (images, target) in enumerate(test_loader):
        target = torch.eye(10).index_select(dim=0, index=target)

        batch_size = images.size(0)

        if TRAIN_ON_GPU:
            images, target = images.cuda(), target.cuda()

        # forward pass: compute predicted outputs by passing inputs to the model
        caps_output, reconstructions, y = capsule_net()
        # calculate the loss
        loss = criterion(caps_output, target, images, reconstructions)
        # update average test loss 
        test_loss += loss.item()
        # convert output probabilities to predicted class
        _, pred = torch.max(y.data.cpu(), 1)
        _, target_shape = torch.max(target.data.cpu(), 1)

        # compare predictions to true label
        correct = np.squeeze(pred.eq(target_shape.data.view_as(pred)))
        # calculate test accuracy for each object class
        for i in range(batch_size):
            label = target_shape.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # avg test loss
    avg_test_loss = test_loss/len(test_loader)
    print('Test Loss: {:.8f}\n'.format(avg_test_loss))

    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                str(i), 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    
    # return last batch of capsule vectors, images, reconstructions
    return caps_output, images, reconstructions

In [169]:
# call test function and get reconstructed images
caps_output, images, reconstructions = test(capsule_net, test_loader)

Test Loss: 0.95494144

Test Accuracy of     0:  0% ( 0/1000)
Test Accuracy of     1:  0% ( 0/1000)
Test Accuracy of     2: 94% (945/1000)
Test Accuracy of     3:  0% ( 0/1000)
Test Accuracy of     4:  0% ( 0/1000)
Test Accuracy of     5:  0% ( 0/1000)
Test Accuracy of     6:  4% (47/1000)
Test Accuracy of     7:  0% ( 0/1000)
Test Accuracy of     8:  0% ( 0/1000)
Test Accuracy of     9:  0% ( 0/1000)

Test Accuracy (Overall):  9% (992/10000)


In [182]:
from PIL import Image
def display_images(images, reconstructions):
    '''Plot one row of original CIFAR10 images and another row (below) 
       of their reconstructions.'''
    # convert to numpy images
    images = images.data.cpu().numpy()
    reconstructions = reconstructions.view(-1, images.shape[1], images.shape[2], images.shape[3])
    reconstructions = reconstructions.data.cpu().numpy()
    
    # # plot the first ten input images and then reconstructed images
    # fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(26,5))

    # # input images on top row, reconstructions on bottom
    # for images, row in zip([images, reconstructions], axes):
    #     for img, ax in zip(images, row):
    #         ax.imshow(np.squeeze(img), cmap='gray')
    #         ax.get_xaxis().set_visible(False)
    #         ax.get_yaxis().set_visible(False)

    for i in range(10):
        # plt.imshow(images[i].reshape(32,32,3))
        # plt.imshow(reconstructions[i].reshape(32,32,3))
        img = Image.fromarray(images[i], 'RGB')
        img = Image.fromarray(reconstructions[i], 'RGB')

        # img.save('my.png')
        img.show()
    # plt.show()

In [183]:
# display original and reconstructed images, in rows
display_images(images, reconstructions)

In [ ]:
# convert data to Tensor *and* perform random affine transformation
transform = transforms.Compose(
    [transforms.RandomAffine(degrees=30, translate=(0.1,0.1)),
     transforms.ToTensor()]
    )

# test dataset
transformed_test_data = datasets.CIFAR10(root='data', train=False,
                                       download=True, transform=transform)

# prepare data loader
transformed_test_loader = torch.utils.data.DataLoader(transformed_test_data, 
                                                      batch_size=batch_size,
                                                      num_workers=num_workers)

In [ ]:
# obtain one batch of test images
dataiter = iter(transformed_test_loader)
images, labels = dataiter.next()
images = images.numpy()

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

In [ ]:
# call test function and get reconstructed images
_, images, reconstructions = test(capsule_net, transformed_test_loader)

In [ ]:
# original input images
display_images(images, reconstructions)

In [ ]:
def vector_analysis(capsule_net, x, select_idx=1):
    '''Generates perturbed iage reconstructions given some digit capsule outputs.
       param capsule_net: trained capsule network
       param x: a batch of digit capsule outputs
       param select_idx: selects which image in a batch to analyze, default = 1 
       return: list of perturbed, reconstructed images
       '''
    
    classes = (x ** 2).sum(dim=-1) ** 0.5
    classes = F.softmax(classes, dim=-1)

    # find the capsule with the maximum vector length
    # here, vector length indicates the probability of a class' existence
    _, max_length_indices = classes.max(dim=1)

    # create a sparse class matrix
    sparse_matrix = torch.eye(10) # 10 is the number of classes
    if TRAIN_ON_GPU:
        sparse_matrix = sparse_matrix.cuda()
    # get the class scores from the "correct" capsule
    y = sparse_matrix.index_select(dim=0, index=max_length_indices.data)

    # create reconstructed pixels
    x = x * y[:, :, None]
    
    # flatten image into a vector shape (batch_size, vector_dim)
    flattened_x = x.reshape(x.size(0), -1)
    # select a single image from a batch to work with
    flattened_x = flattened_x[select_idx]
    
    # track reconstructed images
    reconstructed_ims = []
    # values to change *one* vector dimension by
    perturb_range = np.arange(-0.25, 0.30, 0.05)
    
    # iterate through 16 vector dims
    for k in range(16):
        # create a copy of flattened_x to modify
        transformed_x = None

        if TRAIN_ON_GPU:
            transformed_x = torch.zeros(*flattened_x.size()).cuda()
        else:
            transformed_x = torch.zeros(*flattened_x.size())
            
        transformed_x[:] = flattened_x[:]
        # iterate through each perturbation value
        for j in range(len(perturb_range)):
            # for each capsule output
            for i in range(10):
                transformed_x[k+(16*i)] = flattened_x[k+(16*i)]+perturb_range[j]

            # create reconstructed images
            reconstructions = capsule_net.decoder.linear_layers(transformed_x)
            # reshape into 28x28 image, (batch_size, depth, x, y)
            reconstructions = reconstructions.view(-1, 1, 28, 28)
            reconstructed_ims.append(reconstructions)
    
    # return final list of reconstructed ims    
    return reconstructed_ims
        

In [ ]:
# call function and get perturbed reconstructions
reconstructed_ims = vector_analysis(capsule_net, caps_output, select_idx=1)

In [ ]:
fig = plt.figure(figsize=(10, 20)) # define figsize

# display all ims
for idx in range(len(reconstructed_ims)):
    # convert to numpy images
    image = reconstructed_ims[idx]
    image = image.detach().cpu().numpy()
    # display 16 rows of images
    ax = fig.add_subplot(16, len(reconstructed_ims)/16, idx+1, xticks=[], yticks=[])
    ax.imshow(image.squeeze(), cmap='gray')
    